<a href="https://colab.research.google.com/github/kcanmersin/Google-Colab-Notebooks/blob/main/spam_email_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [66]:
import gdown
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


In [50]:
df = pd.read_csv("/content/mail_data.csv")
print(df.head())

  Category                                            Message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...


In [51]:
df.describe()

,Category,Message
count,5572,5572
unique,2,5157
top,ham,"Sorry, I'll call later"
freq,4825,30


In [52]:
df.isnull().sum()

Category    0
Message     0
dtype: int64

In [53]:
data = df.where(pd.notnull(df), " ")

In [54]:
data.shape

(5572, 2)

In [55]:
label_encoder = LabelEncoder()
data["Category"] = label_encoder.fit_transform(df["Category"])

print(df)

     Category                                            Message
0         ham  Go until jurong point, crazy.. Available only ...
1         ham                      Ok lar... Joking wif u oni...
2        spam  Free entry in 2 a wkly comp to win FA Cup fina...
3         ham  U dun say so early hor... U c already then say...
4         ham  Nah I don't think he goes to usf, he lives aro...
...       ...                                                ...
5567     spam  This is the 2nd time we have tried 2 contact u...
5568      ham               Will ü b going to esplanade fr home?
5569      ham  Pity, * was in mood for that. So...any other s...
5570      ham  The guy did some bitching but I acted like i'd...
5571      ham                         Rofl. Its true to its name

[5572 rows x 2 columns]


In [56]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [57]:
X= data["Message"]
y=data["Category"]

In [58]:
X

0       Go until jurong point, crazy.. Available only ...
1                           Ok lar... Joking wif u oni...
2       Free entry in 2 a wkly comp to win FA Cup fina...
3       U dun say so early hor... U c already then say...
4       Nah I don't think he goes to usf, he lives aro...
                              ...                        
5567    This is the 2nd time we have tried 2 contact u...
5568                 Will ü b going to esplanade fr home?
5569    Pity, * was in mood for that. So...any other s...
5570    The guy did some bitching but I acted like i'd...
5571                           Rofl. Its true to its name
Name: Message, Length: 5572, dtype: object

In [59]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [60]:
feature_extraction = TfidfVectorizer(min_df=1, stop_words="english", lowercase=True)

X_train_features = feature_extraction.fit_transform(X_train)

X_test_features = feature_extraction.transform(X_test)

y_train = y_train.astype(int)
y_test = y_test.astype(int)


In [61]:
print(X_train_features)

  (0, 4346)	0.3238008504874723
  (0, 2353)	0.2545072203817634
  (0, 6695)	0.3616065178053154
  (0, 5895)	0.415102954745324
  (0, 2719)	0.299459437576315
  (0, 3822)	0.37729728742748087
  (0, 3789)	0.4750235197588447
  (0, 3321)	0.2638802854739516
  (1, 4343)	0.6555659308129219
  (1, 1858)	0.5163195438969705
  (1, 3365)	0.5510421389942982
  (2, 4267)	0.531599749449541
  (2, 6215)	0.43979370278404856
  (2, 6597)	0.40097414833733686
  (2, 5672)	0.32606636481997364
  (2, 3025)	0.3502912545366897
  (2, 3185)	0.3663054742561573
  (3, 4447)	0.18080236341909536
  (3, 3084)	0.14346439189216004
  (3, 2877)	0.30030357190007717
  (3, 5515)	0.16439483489485024
  (3, 6922)	0.13381964389308706
  (3, 7080)	0.19700844583868773
  (3, 5535)	0.30030357190007717
  (3, 7398)	0.19877707762085306
  :	:
  (4454, 397)	0.31848634658760416
  (4454, 4027)	0.2561192223695296
  (4454, 6409)	0.2511086901671169
  (4454, 7382)	0.23350338191116915
  (4454, 4578)	0.28626353932821713
  (4455, 6600)	0.3164025961524856
  (4

In [64]:


svm_classifier = SVC(kernel='linear', C=1, probability=True)
tree_classifier = DecisionTreeClassifier()
rf_classifier = RandomForestClassifier()
logistic_classifier = LogisticRegression()

ensemble_model = VotingClassifier(estimators=[
    ('svm', svm_classifier),
    ('tree', tree_classifier),
    ('rf', rf_classifier),
    ('logistic', logistic_classifier)
], voting='soft')

ensemble_model.fit(X_train_features, y_train)

y_pred = ensemble_model.predict(X_test_features)

accuracy = accuracy_score(y_test, y_pred)
print("Ensemble Model Accuracy:", accuracy)


Ensemble Model Accuracy: 0.9847533632286996


In [65]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, classification_report

y_pred = ensemble_model.predict(X_test_features)

accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

f1 = f1_score(y_test, y_pred)
print("F1 Score:", f1)

recall = recall_score(y_test, y_pred)
print("Recall:", recall)

precision = precision_score(y_test, y_pred)
print("Precision:", precision)

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


Accuracy: 0.9847533632286996
F1 Score: 0.9395017793594307
Recall: 0.8979591836734694
Precision: 0.9850746268656716
Classification Report:
               precision    recall  f1-score   support

           0       0.98      1.00      0.99       968
           1       0.99      0.90      0.94       147

    accuracy                           0.98      1115
   macro avg       0.98      0.95      0.97      1115
weighted avg       0.98      0.98      0.98      1115



In [72]:
input=["FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv"]

input_features=feature_extraction.transform(input)

pred=ensemble_model.predict(input_features)
if( pred[0]==1):
  print("Spam")
else:
  print("Ham")

Spam


In [73]:
input=["Nah I don't think he goes to usf, he lives around here though"]

input_features=feature_extraction.transform(input)

pred=ensemble_model.predict(input_features)
if( pred[0]==1):
  print("Spam")
else:
  print("Ham")

Ham
